In [1]:
#Spotify imports
from dotenv import load_dotenv
import os
import base64
from requests import post, get
import json
import nltk
from nltk.tokenize import word_tokenize
#nltk.download('punkt')

In [4]:
#Initializing API calls using Spotify tokens'

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers = headers, data = data)
    json_result = json.loads(result.content)
    token = json_result['access_token']
    return token 

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}



In [4]:

# Deprecated function, newer function to split string below

def string_split(string):
    if string.find("X") >= 0:
        pos = string.find("X")
        return string[:pos]
    if string.find("With") >= 0:
        pos = string.find("With")
        return string[:pos]
    if string.find("Featuring") >= 0:
        pos = string.find('Featuring')
        return string[:pos]
    if string.find("(") >= 0:
        pos = string.find("(")
        return string[:pos]
    if string.find(",") >= 0:
        pos = string.find(",")
        return string[:pos]
    if string.find("&") >= 0:
        pos = string.find("&")
        return string[:pos]
    if string.find("x") >= 0:
        pos = string.find("x")
        return string[:pos]
    if string.find("/") >= 0:
        pos = string.find("/")
        return string[:pos]
    return string

# So in order to figure out how to gain Artist Full names, we are going to tokenise every artist entry we see first.
# After which, we go through the list of tokens. If we match a token to a list of tokens which indicate that the artist name has ended.

break_characters = ['X', 'x', 'With', 'Featuring', '(', ')', ',', '&', '/', '"', 'Duet', 'The']

def list_of_artists(artists):
    artist_list = list()
    tokens = word_tokenize(artists)
    artist = ''
    for token in tokens:
        if token in break_characters:
            if artist != '':
                artist_list.append(artist)
                artist = ''
        else:
            artist += ' ' + token
    if artist != '':
        artist_list.append(artist)
    return artist_list

# Now we have a list of artists based off our splits

#Testing code
    
# print(string_split("Blessd, Justin Quiles & Lenny Tavarez"))

print('Metro Boomin, Future & Chris Brown')
print(word_tokenize('Metro Boomin, Future & Chris Brown'))
print(list_of_artists('Metro Boomin, Future & Chris Brown'))

#word_tokenize('"Carolina Gaitan, Mauro Castillo, Adassa, Rhenzy Feliz, Diane Guerrero, Stephanie Beatriz & Encanto Cast"')

# word_tokenize('Silk Sonic (Bruno Mars & Anderson .Paak)')

Metro Boomin, Future & Chris Brown
['Metro', 'Boomin', ',', 'Future', '&', 'Chris', 'Brown']
[' Metro Boomin', ' Future', ' Chris Brown']


In [6]:

# here we have obtained the token we can use to get the attributes we want from our songs
token = get_token()

#search for the track on spotify api
def search_for_track(token, track_name, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q=artist:{artist_name}&track:{track_name}&type=track&limit=1"

    query_url = url + query
    result = get(query_url, headers = headers)
    json_result = json.loads(result.content)
    return json_result

def search_for_track(token, track_name, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q=artist:{artist_name}&track:{track_name}&type=track&limit=1"

    query_url = url + query
    result = get(query_url, headers = headers)
    json_result = json.loads(result.content)
    return json_result


#slice the track call to get track id
def get_track_id(track_dict):
    return track_dict['tracks']['items'][0]['id']

#extract track features 
def get_track_features(token, track_id):
    url = f"https://api.spotify.com/v1/audio-features/{track_id}?country=US"
    headers = get_auth_header(token)
    #query = f"?{track_id}"

    query_url = url #+ query
    result = get(query_url, headers = headers)
    json_result = json.loads(result.content)
    return json_result

def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q=artist:{artist_name}&type=artist&limit=1"

    query_url = url + query
    result = get(query_url, headers = headers)
    json_result = json.loads(result.content)
    return json_result

def get_artist_score(artist_dict):
    return artist_dict['artists']['items'][0]['popularity']

def get_artist_id(artist_dict):
    return artist_dict['artists']['items'][0]['id']

#testing functions

#print(search_for_track(token,"ghost","Justin")['tracks']['items'][0]['id'])
#
#print(type(get_track_features(token, "6I3mqTwhRpn34SLVafSH7G")))

# track = search_for_track(token, 'do it to it', 'acraze')

# artist = search_for_artist(token, '!')
# track = search_for_track(token, 'Enemy', 'Imagine Dragons')
# track_id = get_track_id(track)
# features = get_track_features(token, track_id)

# print(features)



In [7]:
import numpy as np
import pandas as pd

webscrape = pd.read_csv('billboard.csv')

webscrape = webscrape.drop(columns = 'Unnamed: 0')

# Adding rows of features we want to collect from spotify

webscrape['Danceability'] = np.nan
webscrape['Energy'] = np.nan
webscrape['Key'] = np.nan
webscrape['Loudness'] = np.nan
webscrape['Mode'] = np.nan
webscrape['Speechiness'] = np.nan
webscrape['Acousticness'] = np.nan
webscrape['Instrumentalness'] = np.nan
webscrape['Liveness'] = np.nan
webscrape['Valence'] = np.nan
webscrape['Tempo'] = np.nan
webscrape['Duration ms'] = np.nan
webscrape['Time Signature'] = np.nan
webscrape['Artist Score'] = np.nan

webscrape.head()


,id,Song title,Artist,Current Ranking,Number of weeks on billboard,Peak Ranking,Date,Danceability,Energy,Key,...,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration ms,Time Signature,Artist Score
0,1,All I Want For Christmas Is You,Mariah Carey,1,17,1,2022-01-01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Last Christmas,Wham!,2,15,2,2022-01-01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Rockin' Around The Christmas Tree,Brenda Lee,3,13,3,2022-01-01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Jingle Bell Rock,Bobby Helms,4,13,4,2022-01-01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,abcdefu,GAYLE,5,6,2,2022-01-01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# We create a dictionary of (song, artist) to ensure that we do not need to call the spotify api again if the song was already requested before

song_dict = dict()


for index, row in webscrape.iterrows():
    #artist = string_split(row['Artist'])
    artist_list = list_of_artists(row['Artist'])
    if (row['Song title'], artist_list[0]) not in song_dict.keys():
        track = search_for_track(token, row['Song title'], artist_list[0])
        song_dict[(row['Song title'], artist_list[0])] = {}
        try:
            track_id = get_track_id(track)
            features = get_track_features(token, track_id)
            webscrape.loc[index, 'Danceability'] = features['danceability']
            song_dict[(row['Song title'], artist_list[0])]['Danceability'] = features['danceability']
            webscrape.loc[index,'Energy'] = features['energy']
            song_dict[(row['Song title'], artist_list[0])]['Energy'] = features['energy']
            webscrape.loc[index, 'Key'] = features['key']
            song_dict[(row['Song title'], artist_list[0])]['Key'] = features['key']
            webscrape.loc[index, 'Loudness'] = features['loudness']
            song_dict[(row['Song title'], artist_list[0])]['Loudness'] = features['loudness']
            webscrape.loc[index, 'Mode'] = features['mode']
            song_dict[(row['Song title'], artist_list[0])]['Mode'] = features['mode']
            webscrape.loc[index, 'Speechiness'] = features['speechiness']
            song_dict[(row['Song title'], artist_list[0])]['Speechiness'] = features['speechiness']
            webscrape.loc[index, 'Acousticness'] = features['acousticness']
            song_dict[(row['Song title'], artist_list[0])]['Acousticness'] = features['acousticness']
            webscrape.loc[index, 'Instrumentalness'] = features['instrumentalness']
            song_dict[(row['Song title'], artist_list[0])]['Instrumentalness'] = features['instrumentalness']
            webscrape.loc[index, 'Liveness'] = features['liveness']
            song_dict[(row['Song title'], artist_list[0])]['Liveness'] = features['liveness']
            webscrape.loc[index, 'Valence'] = features['valence']
            song_dict[(row['Song title'], artist_list[0])]['Valence'] = features['valence']
            webscrape.loc[index, 'Tempo'] = features['tempo']
            song_dict[(row['Song title'], artist_list[0])]['Tempo'] = features['tempo']
            webscrape.loc[index, 'Duration ms'] = features['duration_ms']
            song_dict[(row['Song title'], artist_list[0])]['Duration ms'] = features['duration_ms']
            webscrape.loc[index, 'Time Signature'] = features['time_signature']
            song_dict[(row['Song title'], artist_list[0])]['Time Signature'] = features['time_signature']

        except:
            print('song api error at ' +  str(index))
            print(row['Artist'])
        song_score = 0 #to represent the artists (calculated by mean of artist scores)
        no_of_artists = 0
        artist_id_set = set() #to ensure that if the string split retrieves the same artist, we do not add their score again
        for artist in artist_list:
            if artist in artist_id_set:
                continue
            try:
                artist_info = search_for_artist(token, artist)
                artist_score = get_artist_score(artist_info)
                artist_id = get_artist_id(artist_info)
                artist_id_set.add(artist_id)
                song_score += artist_score
                no_of_artists += 1 #to ensure only artists whose scores can be retrieved are calculated
            except:
                print('artist api error at ' +  str(index))
                print(artist)
        webscrape.loc[index, 'Artist Score'] = song_score / no_of_artists
        song_dict[(row['Song title'], artist_list[0])]['Artist Score'] = song_score / no_of_artists
    else:
        #print("I'm not useless!")    
        webscrape.loc[index, 'Danceability'] = song_dict[(row['Song title'], artist_list[0])]['Danceability'] 
        webscrape.loc[index,'Energy'] = song_dict[(row['Song title'], artist_list[0])]['Energy']
        webscrape.loc[index, 'Key'] = song_dict[(row['Song title'], artist_list[0])]['Key']
        webscrape.loc[index, 'Loudness'] = song_dict[(row['Song title'], artist_list[0])]['Loudness']
        webscrape.loc[index, 'Mode'] = song_dict[(row['Song title'], artist_list[0])]['Mode']
        webscrape.loc[index, 'Speechiness'] = song_dict[(row['Song title'], artist_list[0])]['Speechiness']
        webscrape.loc[index, 'Acousticness'] = song_dict[(row['Song title'], artist_list[0])]['Acousticness']
        webscrape.loc[index, 'Instrumentalness'] = song_dict[(row['Song title'], artist_list[0])]['Instrumentalness'] 
        webscrape.loc[index, 'Liveness'] = song_dict[(row['Song title'], artist_list[0])]['Liveness']
        webscrape.loc[index, 'Valence'] = song_dict[(row['Song title'], artist_list[0])]['Valence']
        webscrape.loc[index, 'Tempo'] = song_dict[(row['Song title'], artist_list[0])]['Tempo']
        webscrape.loc[index, 'Duration ms'] = song_dict[(row['Song title'], artist_list[0])]['Duration ms']
        webscrape.loc[index, 'Time Signature'] = song_dict[(row['Song title'], artist_list[0])]['Time Signature']
        webscrape.loc[index, 'Artist Score'] = song_dict[(row['Song title'], artist_list[0])]['Artist Score']


In [9]:
webscrape.to_csv("full_data.csv")


In [10]:
webscrape.head()

,id,Song title,Artist,Current Ranking,Number of weeks on billboard,Peak Ranking,Date,Danceability,Energy,Key,...,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration ms,Time Signature,Artist Score
0,1,All I Want For Christmas Is You,Mariah Carey,1,17,1,2022-01-01,0.837,0.462,0.0,...,1.0,0.0601,0.0339,0.000000,0.0900,0.762,139.986,201400.0,4.0,79.0
1,2,Last Christmas,Wham!,2,15,2,2022-01-01,0.735,0.478,2.0,...,1.0,0.0293,0.1890,0.000002,0.3550,0.947,107.682,262960.0,4.0,13.0
2,3,Rockin' Around The Christmas Tree,Brenda Lee,3,13,3,2022-01-01,0.589,0.472,8.0,...,1.0,0.0502,0.6140,0.000000,0.5050,0.898,67.196,126267.0,4.0,58.0
3,4,Jingle Bell Rock,Bobby Helms,4,13,4,2022-01-01,0.754,0.424,2.0,...,1.0,0.0363,0.6430,0.000000,0.0652,0.806,119.705,130973.0,4.0,50.0
4,5,abcdefu,GAYLE,5,6,2,2022-01-01,0.695,0.540,4.0,...,1.0,0.0493,0.2990,0.000000,0.3670,0.415,121.932,168602.0,4.0,69.0
